# **PR: weekly data**

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
from pandas.tseries.offsets import *

In [2]:
# download price data from Yahoo finance

p_intel = yf.download('AAPL',start = '2007-01-01')
p_google = yf.download('JNJ',start = '2007-01-01')
 
# merge the tow tables above

p_intel = p_intel[['Adj Close']].rename(columns = {'Adj Close':'Close_Intel'})
p_google = p_google[['Adj Close']].rename(columns = {'Adj Close':'Close_Google'})
 
price = pd.concat([p_intel, p_google], axis=1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
price = price.reset_index()

In [4]:
price

,Date,Close_Intel,Close_Google
0,2007-01-03,2.558946,42.234432
1,2007-01-04,2.615744,42.762379
2,2007-01-05,2.597117,42.374374
3,2007-01-08,2.609942,42.304405
4,2007-01-09,2.826749,42.145390
...,...,...,...
3883,2022-06-06,146.139999,176.399994
3884,2022-06-07,148.710007,178.339996
3885,2022-06-08,147.960007,177.279999
3886,2022-06-09,142.639999,173.710007


In [5]:
# get all days within the sample period

start_date = price['Date'].astype(str).iloc[0]
end_date = price['Date'].astype(str).iloc[-1]

In [6]:
date_all = pd.date_range(start_date, end_date, freq='D').to_frame().rename(columns={0:'Date'}).reset_index(drop=True)

In [7]:
# merge with price data

price_all = pd.merge(date_all,price,how='left')

In [8]:
price_all

,Date,Close_Intel,Close_Google
0,2007-01-03,2.558946,42.234432
1,2007-01-04,2.615744,42.762379
2,2007-01-05,2.597117,42.374374
3,2007-01-06,NaN,NaN
4,2007-01-07,NaN,NaN
...,...,...,...
5633,2022-06-06,146.139999,176.399994
5634,2022-06-07,148.710007,178.339996
5635,2022-06-08,147.960007,177.279999
5636,2022-06-09,142.639999,173.710007


In [9]:
# the 'ffill' method propagates last valid observation forward

price_all = price_all.fillna(method='ffill')

In [10]:
price_all

,Date,Close_Intel,Close_Google
0,2007-01-03,2.558946,42.234432
1,2007-01-04,2.615744,42.762379
2,2007-01-05,2.597117,42.374374
3,2007-01-06,2.597117,42.374374
4,2007-01-07,2.597117,42.374374
...,...,...,...
5633,2022-06-06,146.139999,176.399994
5634,2022-06-07,148.710007,178.339996
5635,2022-06-08,147.960007,177.279999
5636,2022-06-09,142.639999,173.710007


In [11]:
# generate day of week variables

price_all['Day_Name'] = price_all['Date'].dt.day_name()

In [12]:
price_all

,Date,Close_Intel,Close_Google,Day_Name
0,2007-01-03,2.558946,42.234432,Wednesday
1,2007-01-04,2.615744,42.762379,Thursday
2,2007-01-05,2.597117,42.374374,Friday
3,2007-01-06,2.597117,42.374374,Saturday
4,2007-01-07,2.597117,42.374374,Sunday
...,...,...,...,...
5633,2022-06-06,146.139999,176.399994,Monday
5634,2022-06-07,148.710007,178.339996,Tuesday
5635,2022-06-08,147.960007,177.279999,Wednesday
5636,2022-06-09,142.639999,173.710007,Thursday


In [13]:
# get Monday prices

price_mon = price_all[price_all['Day_Name']=='Monday']

In [14]:
price_mon

,Date,Close_Intel,Close_Google,Day_Name
5,2007-01-08,2.609942,42.304405,Monday
12,2007-01-15,2.889350,42.387100,Monday
19,2007-01-22,2.650250,42.730572,Monday
26,2007-01-29,2.624295,42.075439,Monday
33,2007-02-05,2.563221,42.170841,Monday
...,...,...,...,...
5605,2022-05-09,152.059998,176.197769,Monday
5612,2022-05-16,145.539993,176.942978,Monday
5619,2022-05-23,143.110001,179.440002,Monday
5626,2022-05-30,149.639999,181.089996,Monday


In [15]:
def date_function(x):
    result = str(x) # numerical type, not string type -> indexing is not applied
    return result[0:4] + '-' + result[5:7] + '-' + result[8:10]

price_mon['date'] = pd.DataFrame(price_mon['Date'].apply(date_function))

C:\Users\geodo\AppData\Local\Temp\ipykernel_4468\2191040034.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_mon['date'] = pd.DataFrame(price_mon['Date'].apply(date_function))


In [16]:
price_mon

,Date,Close_Intel,Close_Google,Day_Name,date
5,2007-01-08,2.609942,42.304405,Monday,2007-01-08
12,2007-01-15,2.889350,42.387100,Monday,2007-01-15
19,2007-01-22,2.650250,42.730572,Monday,2007-01-22
26,2007-01-29,2.624295,42.075439,Monday,2007-01-29
33,2007-02-05,2.563221,42.170841,Monday,2007-02-05
...,...,...,...,...,...
5605,2022-05-09,152.059998,176.197769,Monday,2022-05-09
5612,2022-05-16,145.539993,176.942978,Monday,2022-05-16
5619,2022-05-23,143.110001,179.440002,Monday,2022-05-23
5626,2022-05-30,149.639999,181.089996,Monday,2022-05-30


In [17]:
# remove the day name, date columns

price_mon = price_mon.drop(['Day_Name'],axis=1)
price_mon = price_mon.drop(['Date'],axis=1)

In [18]:
price_mon

,Close_Intel,Close_Google,date
5,2.609942,42.304405,2007-01-08
12,2.889350,42.387100,2007-01-15
19,2.650250,42.730572,2007-01-22
26,2.624295,42.075439,2007-01-29
33,2.563221,42.170841,2007-02-05
...,...,...,...
5605,152.059998,176.197769,2022-05-09
5612,145.539993,176.942978,2022-05-16
5619,143.110001,179.440002,2022-05-23
5626,149.639999,181.089996,2022-05-30


In [19]:
# set index

price_mon = price_mon.set_index(['date'])

In [20]:
price_mon

,Close_Intel,Close_Google
date,,
2007-01-08,2.609942,42.304405
2007-01-15,2.889350,42.387100
2007-01-22,2.650250,42.730572
2007-01-29,2.624295,42.075439
2007-02-05,2.563221,42.170841
...,...,...
2022-05-09,152.059998,176.197769
2022-05-16,145.539993,176.942978
2022-05-23,143.110001,179.440002


In [21]:
# monday close-to-monday close weekly returns

week_mon_ret = price_mon.pct_change(1).dropna()

In [22]:
week_mon_ret

,Close_Intel,Close_Google
date,,
2007-01-15,0.107055,0.001955
2007-01-22,-0.082752,0.008103
2007-01-29,-0.009794,-0.015332
2007-02-05,-0.023272,0.002267
2007-02-12,0.011199,-0.012670
...,...,...
2022-05-09,-0.035937,-0.007333
2022-05-16,-0.042878,0.004229
2022-05-23,-0.016696,0.014112


In [23]:
week_mon_ret.to_excel(excel_writer='weekly_return.xlsx')

## **references**

https://invest-in-yourself.tistory.com/296